In [3]:
import os

In [1]:
%pwd

'c:\\Users\\asus\\onedrive\\desktop\\Machine Learning project with Mlflow\\Machine-Learning-project-with-Mlflow\\research'

In [4]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\asus\\onedrive\\desktop\\Machine Learning project with Mlflow\\Machine-Learning-project-with-Mlflow\\research'

In [8]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Anushkaaaaa27/Machine-Learning-Project-with-MLflow.mlflow"

os.environ["MLFLOW_TRACKING_USERNAME"]="Anushkaaaaa27"

os.environ["MLFLOW_TRACKING_PASSWORD"]="da380ca37be9250342fc2da07b1ae830b77be2f5"


In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: str
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/Anushkaaaaa27/Machine-Learning-Project-with-MLflow.mlflow"
        )

        return model_evaluation_config

In [12]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            #Saving matrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metrics("rmse", rmse)
            mlflow.log_metrics("r2", r2)
            mlflow.log_metrics("mae", mae)


            #Model registry does not work with file store
            if tracking_url_type_store != "file":
                #Register the model
                



